In [1]:
import tensorflow as tf
from moviepy.editor import VideoFileClip
import numpy as np
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.layers import LSTM, TimeDistributed, Dense
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input
import cv2
import random
from tqdm import tqdm
from pathlib import Path
from glob import glob
import os
import re
import pickle

tqdm.pandas()

2023-02-02 10:32:17.085495: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from tensorflow.keras.models import load_model
import tensorflow as tf


physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)


2023-02-02 10:32:19.065105: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-02-02 10:32:19.199432: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-02-02 10:32:19.199511: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-02-02 10:32:19.204811: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-02-02 10:32:19.204890: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2023-02-02 10:32:19.206066: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.

In [3]:
# !pip3 install fvcore

In [4]:
# !pip3 install pytorchvideo

In [5]:
# import torch
# # Choose the `slow_r50` model 

# model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50', pretrained=True)

In [6]:
# import json
# import urllib
# from pytorchvideo.data.encoded_video import EncodedVideo

# from torchvision.transforms import Compose, Lambda
# from torchvision.transforms._transforms_video import (
#     CenterCropVideo,
#     NormalizeVideo,
# )
# from pytorchvideo.transforms import (
#     ApplyTransformToKey,
#     ShortSideScale,
#     UniformTemporalSubsample
# )

In [7]:
# # Set to GPU or CPU
# device = "cpu"
# model = model.eval()
# model = model.to(device)

In [8]:
# json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
# json_filename = "kinetics_classnames.json"
# try: urllib.URLopener().retrieve(json_url, json_filename)
# except: urllib.request.urlretrieve(json_url, json_filename)

In [9]:
# with open(json_filename, "r") as f:
#     kinetics_classnames = json.load(f)

# # Create an id to label name mapping
# kinetics_id_to_classname = {}
# for k, v in kinetics_classnames.items():
#     kinetics_id_to_classname[v] = str(k).replace('"', "")

In [10]:
# side_size = 256
# mean = [0.45, 0.45, 0.45]
# std = [0.225, 0.225, 0.225]
# crop_size = 256
# num_frames = 8
# sampling_rate = 8
# frames_per_second = 30

# # Note that this transform is specific to the slow_R50 model.
# transform =  ApplyTransformToKey(
#     key="video",
#     transform=Compose(
#         [
#             UniformTemporalSubsample(num_frames),
#             Lambda(lambda x: x/255.0),
#             NormalizeVideo(mean, std),
#             ShortSideScale(
#                 size=side_size
#             ),
#             CenterCropVideo(crop_size=(crop_size, crop_size))
#         ]
#     ),
# )

# # The duration of the input clip is also specific to the model.
# clip_duration = (num_frames * sampling_rate)/frames_per_second

In [11]:
# url_link = "https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4"
# video_path = 'archery.mp4'
# try: urllib.URLopener().retrieve(url_link, video_path)
# except: urllib.request.urlretrieve(url_link, video_path)

In [12]:
# video_path = "/mount1/mount1/First-Impression/Apparent_traits/DeepImpression/deep_impression/_fJcW5234bY.001.mp4"

In [13]:
# # Select the duration of the clip to load by specifying the start and end duration
# # The start_sec should correspond to where the action occurs in the video
# start_sec = 0
# end_sec = start_sec + clip_duration

# # Initialize an EncodedVideo helper class and load the video
# video = EncodedVideo.from_path(video_path)

# # Load the desired clip
# video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)


In [14]:
# video_data

In [15]:

# # Apply a transform to normalize the video input
# video_data = transform(video_data)

# # Move the inputs to the desired device
# inputs = video_data["video"]
# inputs = inputs.to(device)

In [16]:
# # Pass the input clip through the model
# preds = model(inputs[None, ...])

# # Get the predicted classes
# post_act = torch.nn.Softmax(dim=1)
# preds = post_act(preds)
# pred_classes = preds.topk(k=5).indices[0]

# # Map the predicted classes to the label names
# pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes]
# print("Top 5 predicted labels: %s" % ", ".join(pred_class_names))

In [17]:
# import cv2

# cnt  =0
# cap = cv2.VideoCapture(video_path)
# while True:
#     ret, fr = cap.read()
#     if ret:
#         cnt += 1
#     else:
#         break

In [18]:
# cnt / 30

In [19]:
# !pip3 install tensorflow

In [20]:
base_model = ResNet101V2(include_top=False, weights="imagenet",input_shape=(224,224, 3), pooling='max')
# base_model.trainble = False


2023-02-02 10:32:19.592761: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-02 10:32:19.759975: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: NVIDIA A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-02-02 10:32:19.764827: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2023-02-02 10:32:19.764943: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-02-02 10:32:25.914858: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnec

In [21]:
# model.summary()

In [22]:


emb = []

In [23]:
def preprocess(video_pth, seq_len=30, target_resolution=(224, 224)):
    """
    extract frames and audio from the video,
    store the cropped frames and audio file in the output folders
    seq_len: how many frames will be extracted from the video.
              Considering all videos from this dataset have similar duration
              video_duration = seq_len / fps
    target_resolution: (desired_height, desired_width) of the facial frame extracted
    """
    frames = []
    video = VideoFileClip(video_pth, target_resolution=target_resolution)
    
    times = list(np.arange(0, video.duration, video.duration/seq_len))
    if len(times) < seq_len:
        times.append(video.duration)
    times = times[:seq_len]

    for i, t in enumerate(times):
        img = cv2.cvtColor(video.get_frame(t), cv2.COLOR_BGR2RGB)
        frames.append(img)
        img1 = np.expand_dims(img, 0)
        # print(img1.shape)
        em = base_model.predict(img1)
        emb.append(em)
    print("Video duration {} seconds. Extracted {} frames".format(video.duration, len(times)))
    
# preprocess(video_path)

In [24]:
np.array(emb).shape

(0,)

In [25]:
time_step = 30
h = w =224
c = 3

In [26]:
# TimeDistributed Layer is used to run resnet model first on all the frames and then run LSTM on top of it
input_layer_env = Input(shape=(time_step, h, w, c))

tm_layer_env = TimeDistributed(base_model)(input_layer_env)

lstm_env = LSTM(256)(tm_layer_env)

op_env =  Dense(5, activation='sigmoid')(lstm_env)

model_env = Model(input_layer_env, op_env)

In [27]:
import tensorflow.keras.backend as k
def mean_acc(y_true, y_pred):
    diff = k.abs(y_true - y_pred)
    return k.mean(1-diff)

In [28]:
model_env.compile(
    loss = ['mae'],
    # loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), 
    metrics = [mean_acc]
)

In [29]:
model_env.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 30, 224, 224, 3)] 0         
_________________________________________________________________
time_distributed (TimeDistri (None, 30, 2048)          42626560  
_________________________________________________________________
lstm (LSTM)                  (None, 256)               2360320   
_________________________________________________________________
dense (Dense)                (None, 5)                 1285      
Total params: 44,988,165
Trainable params: 44,890,501
Non-trainable params: 97,664
_________________________________________________________________


# Dataset preperation

## Training data prep

In [30]:
def get_train_file_path(fl):
    train_fol = "/mnt/FirstImpression/unzippedData/**/*.mp4"
    train_fol = glob(train_fol)
    train_fol = [i for i in train_fol if "training" in Path(i).parts[-2]]
    fil = list(filter(lambda x: fl in x, train_fol))
    return fil[0]
#     for i in train_fol:
#         if fl in i:
#             fil = i
            
#     return fil


In [31]:
import pandas as pd

train_annot = pd.read_pickle("/mnt/FirstImpression/annotation_training.pkl")


In [32]:
train_annotation = pd.DataFrame().from_dict(train_annot)

In [33]:
train_annotation['file_name'] = train_annotation.index

In [34]:
train_annotation

,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,file_name
J4GQm9j0JZ0.003.mp4,0.523364,0.552083,0.626374,0.601942,0.504673,0.488889,J4GQm9j0JZ0.003.mp4
zEyRyTnIw5I.005.mp4,0.345794,0.375000,0.472527,0.582524,0.457944,0.366667,zEyRyTnIw5I.005.mp4
nskJh7v6v1U.004.mp4,0.252336,0.291667,0.406593,0.485437,0.373832,0.511111,nskJh7v6v1U.004.mp4
6wHQsN5g2RM.000.mp4,0.457944,0.489583,0.505495,0.398058,0.457944,0.377778,6wHQsN5g2RM.000.mp4
dQOeQYWIgm8.000.mp4,0.607477,0.489583,0.406593,0.621359,0.570093,0.622222,dQOeQYWIgm8.000.mp4
...,...,...,...,...,...,...,...
Eh7WRYXVh9M.000.mp4,0.523364,0.479167,0.626374,0.621359,0.588785,0.544444,Eh7WRYXVh9M.000.mp4
2q8orkMs2Jg.003.mp4,0.728972,0.760417,0.582418,0.524272,0.616822,0.822222,2q8orkMs2Jg.003.mp4
F1lAPYh4t3U.000.mp4,0.700935,0.770833,0.747253,0.699029,0.691589,0.788889,F1lAPYh4t3U.000.mp4
cxJ0u6r0-pU.001.mp4,0.317757,0.531250,0.582418,0.679612,0.616822,0.588889,cxJ0u6r0-pU.001.mp4


In [35]:
# train_annotation['file_path'] = train_annotation['file_name'].progress_apply(lambda x: get_train_file_path(x))

In [36]:
# train_annotation.to_csv("first_impression_train.csv", index=False)

In [37]:
train_annotation = pd.read_csv("first_impression_train.csv")

In [38]:
train_annotation

,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,file_name,file_path
0,0.523364,0.552083,0.626374,0.601942,0.504673,0.488889,J4GQm9j0JZ0.003.mp4,/mnt/FirstImpression/unzippedData/training80_6...
1,0.345794,0.375000,0.472527,0.582524,0.457944,0.366667,zEyRyTnIw5I.005.mp4,/mnt/FirstImpression/unzippedData/training80_7...
2,0.252336,0.291667,0.406593,0.485437,0.373832,0.511111,nskJh7v6v1U.004.mp4,/mnt/FirstImpression/unzippedData/training80_5...
3,0.457944,0.489583,0.505495,0.398058,0.457944,0.377778,6wHQsN5g2RM.000.mp4,/mnt/FirstImpression/unzippedData/training80_3...
4,0.607477,0.489583,0.406593,0.621359,0.570093,0.622222,dQOeQYWIgm8.000.mp4,/mnt/FirstImpression/unzippedData/training80_7...
...,...,...,...,...,...,...,...,...
5995,0.523364,0.479167,0.626374,0.621359,0.588785,0.544444,Eh7WRYXVh9M.000.mp4,/mnt/FirstImpression/unzippedData/training80_5...
5996,0.728972,0.760417,0.582418,0.524272,0.616822,0.822222,2q8orkMs2Jg.003.mp4,/mnt/FirstImpression/unzippedData/training80_5...
5997,0.700935,0.770833,0.747253,0.699029,0.691589,0.788889,F1lAPYh4t3U.000.mp4,/mnt/FirstImpression/unzippedData/training80_6...
5998,0.317757,0.531250,0.582418,0.679612,0.616822,0.588889,cxJ0u6r0-pU.001.mp4,/mnt/FirstImpression/unzippedData/training80_1...


In [39]:
# import pickle as pkl

# with open("./first_impression_train.pkl", 'wb') as f:
#     pkl.dump(train_annot, f)

# # pd.to_pickle("./first_impression_train.pkl")

In [40]:
def get_frames(video_pth, output_pth, seq_len=30, target_resolution=(224, 224)):
    """
    extract frames and audio from the video,
    store the cropped frames and audio file in the output folders
    seq_len: how many frames will be extracted from the video.
              Considering all videos from this dataset have similar duration
              video_duration = seq_len / fps
    target_resolution: (desired_height, desired_width) of the facial frame extracted
    """
    frames = []
    video = VideoFileClip(video_pth, target_resolution=target_resolution)
    
    times = list(np.arange(0, video.duration, video.duration/seq_len))
    if len(times) < seq_len:
        times.append(video.duration)
    times = times[:seq_len]
    
    for i, t in enumerate(times):
        img = cv2.cvtColor(video.get_frame(t), cv2.COLOR_BGR2RGB)
        # frames.append(img)
        # img1 = np.expand_dims(img, 0)
        # print(img1.shape)
        # em = base_model.predict(img1)
        # emb.append(em)
        cv2.imwrite(f"{output_pth}/frame_{i}.jpg", img)
    # print("Video duration {} seconds. Extracted {} frames".format(video.duration, len(times)))

# fet_frames(video_path)

In [41]:
# from glob import glob
# from tqdm import tqdm
# from pathlib import Path
# import os


# # make training folder
# train_folder = "training_frame_folder"

# if not os.path.isdir(train_folder):
#     os.mkdir(train_folder)

# else:
#     train_fol = "/mount1/mount1/First-Impression/unzippedData/**/*.mp4"
#     train_fol = glob(train_fol)
#     train_fol = [i for i in train_fol if "training" in Path(i).parts[-2]]
    
#     for fl in tqdm(train_fol):
#         img_folder = train_folder+"/" + Path(fl).parts[-1].split(".")[0]
        
#         # print(img_folder)
        
#         if not os.path.isdir(img_folder):
#             os.mkdir(img_folder)
#         get_frames(video_pth=fl, output_pth=img_folder)


## val data creation

In [42]:
def get_val_file_path(fl):
    train_fol = "/mnt/FirstImpression/unzippedData/**/*.mp4"
    train_fol = glob(train_fol)
    train_fol = [i for i in train_fol if "validation" in Path(i).parts[-2]]
    fil = list(filter(lambda x: fl in x, train_fol))
    return fil[0]
#     for i in train_fol:
#         if fl in i:
#             fil = i
            
#     return fil


In [43]:
import pandas as pd

val_annot = pd.read_pickle("/mnt/FirstImpression/annotation_validation.pkl")


In [44]:
val_annotation = pd.DataFrame().from_dict(val_annot)

In [45]:
val_annotation['file_name'] = val_annotation.index

In [46]:
# val_annotation['file_path'] = val_annotation['file_name'].progress_apply(lambda x: get_val_file_path(x))

In [47]:
# val_annotation.to_csv("first_impression_val.csv", index=False)

In [48]:
val_annotation = pd.read_csv("first_impression_val.csv")

In [49]:
val_annotation

,extraversion,neuroticism,agreeableness,conscientiousness,interview,openness,file_name,file_path
0,0.644860,0.593750,0.615385,0.640777,0.616822,0.555556,modNfUPt3F4.002.mp4,/mnt/FirstImpression/unzippedData/validation80...
1,0.439252,0.520833,0.417582,0.572816,0.439252,0.411111,h6LOjpCRXtY.005.mp4,/mnt/FirstImpression/unzippedData/validation80...
2,0.457944,0.312500,0.428571,0.398058,0.373832,0.555556,WER4ww680QQ.004.mp4,/mnt/FirstImpression/unzippedData/validation80...
3,0.364486,0.572917,0.527473,0.553398,0.523364,0.322222,c4XnKouozXU.002.mp4,/mnt/FirstImpression/unzippedData/validation80...
4,0.345794,0.468750,0.516484,0.417476,0.383178,0.477778,OEKg-Tvwcbk.002.mp4,/mnt/FirstImpression/unzippedData/validation80...
...,...,...,...,...,...,...,...,...
1995,0.570093,0.614583,0.494505,0.689320,0.626168,0.577778,3LAaFUSGvsU.000.mp4,/mnt/FirstImpression/unzippedData/validation80...
1996,0.542056,0.541667,0.549451,0.669903,0.579439,0.666667,n2BuwHbdilY.000.mp4,/mnt/FirstImpression/unzippedData/validation80...
1997,0.551402,0.593750,0.560440,0.572816,0.504673,0.644444,GcuoyJPO-KU.003.mp4,/mnt/FirstImpression/unzippedData/validation80...
1998,0.514019,0.552083,0.461538,0.572816,0.551402,0.733333,uf_sIIw4zxY.004.mp4,/mnt/FirstImpression/unzippedData/validation80...


In [50]:
# import pickle as pkl

# with open("./first_impression_val.pkl", 'wb') as f:
#     pkl.dump(val_annot, f)


# Custom callback to save model

In [51]:
def generate_output_dir(outdir, run_desc):
    prev_run_dirs = []
    if os.path.isdir(outdir):
        prev_run_dirs = [x for x in os.listdir(outdir) if os.path.isdir(\
            os.path.join(outdir, x))]
    prev_run_ids = [re.match(r'^\d+', x) for x in prev_run_dirs]
    prev_run_ids = [int(x.group()) for x in prev_run_ids if x is not None]
    cur_run_id = max(prev_run_ids, default=-1) + 1
    run_dir = os.path.join(outdir, f'{cur_run_id:05d}-{run_desc}')
    assert not os.path.exists(run_dir)
    os.makedirs(run_dir)
    return run_dir

In [52]:
outdir = "./checkpoint/"
run_desc = "env_personality"

run_dir = generate_output_dir(outdir, run_desc)
print(f"Results saved to: {run_dir}")

Results saved to: ./checkpoint/00005-env_personality


In [53]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

class MyModelCheckpoint(ModelCheckpoint):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    def on_epoch_end(self, epoch, logs=None):
        super().on_epoch_end(epoch,logs)

        # Also save the optimizer state
        filepath = self._get_file_path(epoch=epoch, 
                                       logs=logs, 
                                       # batch=None
                                      )

        filepath = filepath.rsplit( ".", 1 )[ 0 ] 
        filepath += ".pkl"

        with open(filepath, 'wb') as fp:
            pickle.dump(
            {
                'opt': model_env.optimizer.get_config(),
                'epoch': epoch+1,
                'lr': model_env.optimizer.learning_rate
                
             # Add additional keys if you need to store more values
            }, fp, protocol=pickle.HIGHEST_PROTOCOL)
        print('\nEpoch %05d: saving optimizaer to %s' % (epoch + 1, filepath))

In [54]:
from tensorflow.keras.callbacks import ReduceLROnPlateau


# reduce lr on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_mean_acc', factor=0.5, patience=2, 
                                   verbose=1, mode='min', min_lr=0.0000000001)

# Tf dataset creation

In [55]:
import tensorflow as tf


def format_frames(frame, output_size):
    """
        Pad and resize an image from a video.

        Args:
          frame: Image that needs to resized and padded. 
          output_size: Pixel size of the output frame image.

        Return:
          Formatted frame with padding of specified output size.
    """
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return tf.keras.applications.resnet_v2.preprocess_input(frame)
    # return frame

In [56]:
def frames_from_video_file(video_path, n_frames, output_size = (224,224), frame_step = 15):
    """
        Creates frames from each video file present for each category.

        Args:
          video_path: File path to the video.
          n_frames: Number of frames to be created per video file.
          output_size: Pixel size of the output frame image.

        Return:
          An NumPy array of frames in the shape of (n_frames, height, width, channels).
    """
    # Read each video frame by frame
    result = []
    src = cv2.VideoCapture(str(video_path))  

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    need_length = 1 + (n_frames - 1) * frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    # ret is a boolean indicating whether read was successful, frame is the image itself
    ret, frame = src.read()
    result.append(format_frames(frame, output_size))
    for _ in range(n_frames - 1):
        for _ in range(frame_step):
            ret, frame = src.read()
        if ret:
            frame = format_frames(frame, output_size)
            result.append(frame)
        else:
            result.append(np.zeros_like(result[0]))
    src.release()
    result = np.array(result)[..., [2, 1, 0]]
    
    return result

In [57]:
# frames_from_video_file(video_path, 30).shape

In [58]:
# sample df to test model working
df = train_annotation

In [59]:
len(df)

6000

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
train_df, test_df = train_test_split(df, test_size=0.2)

In [62]:
# train_df

In [63]:
train_df.reset_index(inplace=True)

In [64]:
test_df.reset_index(inplace=True)

In [65]:
# def fr_and_label_from_video(video_path,label):
#     """
#         Creates frames from each video file present for each category.

#         Args:
#           video_path: File path to the video.
#           n_frames: Number of frames to be created per video file.
#           output_size: Pixel size of the output frame image.

#         Return:
#           An NumPy array of frames in the shape of (n_frames, height, width, channels).
#     """
    
#     n_frames=30
#     output_size = (224,224)
#     frame_step = 15
    
#     # print(tf.strings.as_string(video_path))
#     print(video_path)
#     print(label)
    
#     # Read each video frame by frame
#     result = []
#     src = cv2.VideoCapture(str(video_path))  
#     print(src.isOpened())
#     video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

#     need_length = 1 + (n_frames - 1) * frame_step

#     if need_length > video_length:
#         start = 0
#     else:
#         max_start = video_length - need_length
#         start = random.randint(0, max_start + 1)

#     src.set(cv2.CAP_PROP_POS_FRAMES, start)
#     # ret is a boolean indicating whether read was successful, frame is the image itself
#     ret, frame = src.read()
#     result.append(format_frames(frame, output_size))
#     for _ in range(n_frames - 1):
#         for _ in range(frame_step):
#             ret, frame = src.read()
#         if ret:
#             frame = format_frames(frame, output_size)
#             result.append(frame)
#         else:
#             result.append(np.zeros_like(result[0]))
#     src.release()
#     result = np.array(result)[..., [2, 1, 0]]
    
#     return result, label
    

In [66]:
# # seperate x_train  and y_train

# y_train = train_df[['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']].values
# x_train = train_df.file_path.values

# tr_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))


In [67]:
train_batch_size = val_batch_size = 8


train_steps = len(train_df)//train_batch_size
val_steps = len(test_df)//val_batch_size

In [68]:
# tf dataset generator
class FrameGenerator:
    def __init__(self, df, n_frames, training = False):
        """ Returns a set of frames with their associated label. 
            Args:
                path: Video file paths.
                n_frames: Number of frames. 
                training: Boolean to determine if training dataset is being created.
        """
        self.df = df
        self.n_frames = n_frames
        self.training = training
  

    def __call__(self):
        if self.training:
            import sklearn
            self.df = sklearn.utils.shuffle(self.df)
            
        for files in self.df.itertuples():
            video_frames = frames_from_video_file(files.file_path, self.n_frames)
            openness = files.openness
            conscientiousness = files.conscientiousness
            extraversion = files.extraversion
            agreeableness = files.agreeableness
            neuroticism = files.neuroticism
            
            label = np.array([files.openness, files.conscientiousness ,files.extraversion, files.agreeableness, files.neuroticism])
            
            yield video_frames, label

In [69]:
# for i in train_ds.take(10):
#     print(i)

In [70]:
# time_steps = 30


# def get_batches(df, b_size = 16, train=False):
#     count = 0 # counter variable 
    
#     if train:
#         import sklearn
#         df = sklearn.utils.shuffle(df)
            
#     # looping over all videos
#     while count < len(df): 
#         all_frames = np.empty((0,time_steps,h,w,c)) # numpy array of batch of videos 
#         labels = np.empty((0,5)) # numpy array to contain the 5 personality scores 

#         if (count + b_size) <= len(df): 
#             batch_size = b_size
#         else: 
#             batch_size = len(df) % b_size # incase when len(data) is not a multiple of batch_size 


#         for i in range(batch_size):
#             # getting names of all image frames in folder for i th video
            
#             ith_val = df.iloc[count]
            
#             f_name = df.iloc[count]['file_path']
                
#             frames_1 = frames_from_video_file(f_name, 30)
#             # stacking all frames in the batch after scaling each frame 
#             all_frames = np.vstack((all_frames,(np.array(frames_1)/255)[np.newaxis,...])) 

#             # extracting the labels
#             l = np.array([ith_val['extraversion'],
#                           ith_val['neuroticism'],
#                           ith_val['agreeableness'],
#                           ith_val['conscientiousness'],
#                           ith_val['openness']
#                          ])
            
#             labels = np.vstack((labels,l[np.newaxis,...]))

#             count += 1

#         # after every epoch the counter must start again 
#         if count >= len(df): 
#             count = 0
#             random.shuffle(df)

#         yield all_frames, labels  # should yield audio and visual input of the model in one list 


In [71]:
# # Making objects for the train and validation generator
# train_batches = get_batches(train_df, b_size = 8, train=True)
# val_batches = get_batches(test_df, b_size = 8)

In [72]:
# train_ds = tf.data.Dataset.from_generator(get_batches(train_df, b_size = 8, train=True))

In [73]:
# train_batches

In [74]:
# x=next(train_batches)

In [75]:
# x[1].shape

In [76]:
# model_env.fit(train_batches, validation_data=val_batches, epochs=20)

In [77]:
# tf dataset generator
class MetaGenerator:
    def __init__(self, df, n_frames, training = False):
        """ Returns a set of frames with their associated label. 
            Args:
                path: Video file paths.
                n_frames: Number of frames. 
                training: Boolean to determine if training dataset is being created.
        """
        self.df = df
        self.n_frames = n_frames
        self.training = training
  

    def __call__(self):
        if self.training:
            import sklearn
            self.df = sklearn.utils.shuffle(self.df)
            
        for files in self.df.itertuples():
            # video_frames = frames_from_video_file(files.file_path, self.n_frames)
            video_frames = files.file_path
            openness = files.openness
            conscientiousness = files.conscientiousness
            extraversion = files.extraversion
            agreeableness = files.agreeableness
            neuroticism = files.neuroticism
            
            label = np.array([files.openness, files.conscientiousness ,files.extraversion, files.agreeableness, files.neuroticism])
            
            yield video_frames, label
            # return video_frames, label

In [78]:
# def frames_from_video_file_map(video_path, label, n_frames, output_size = (224,224), frame_step = 15):
#     """
#         Creates frames from each video file present for each category.

#         Args:
#           video_path: File path to the video.
#           n_frames: Number of frames to be created per video file.
#           output_size: Pixel size of the output frame image.

#         Return:
#           An NumPy array of frames in the shape of (n_frames, height, width, channels).
#     """
#     # Read each video frame by frame
#     result = []
#     src = cv2.VideoCapture(str(video_path))  

#     video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

#     need_length = 1 + (n_frames - 1) * frame_step

#     if need_length > video_length:
#         start = 0
#     else:
#         max_start = video_length - need_length
#         start = random.randint(0, max_start + 1)

#     src.set(cv2.CAP_PROP_POS_FRAMES, start)
#     # ret is a boolean indicating whether read was successful, frame is the image itself
#     ret, frame = src.read()
#     result.append(format_frames(frame, output_size))
#     for _ in range(n_frames - 1):
#         for _ in range(frame_step):
#             ret, frame = src.read()
#         if ret:
#             frame = format_frames(frame, output_size)
#             result.append(frame)
#         else:
#             result.append(np.zeros_like(result[0]))
#     src.release()
#     result = np.array(result)[..., [2, 1, 0]]
    
#     return result, label

In [79]:
from time import time

def frames_from_video_file_map(video_path, label, n_frames, output_size = (224,224), frame_step = 15):
    """
        Creates frames from each video file present for each category.

        Args:
          video_path: File path to the video.
          n_frames: Number of frames to be created per video file.
          output_size: Pixel size of the output frame image.

        Return:
          An NumPy array of frames in the shape of (n_frames, height, width, channels).
    """
    # Read each video frame by frame
    result = []
    src = cv2.VideoCapture(str(video_path))  

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    need_length = 1 + (n_frames - 1) * frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    # ret is a boolean indicating whether read was successful, frame is the image itself
    ret, frame = src.read()
    if ret:
        st = time()
        
        ft_time = time()
        result.append(format_frames(frame, output_size))
    else:
        er_t = time()
        result.append(np.zeros_like(np.random.rand(224, 224, 3)))
        # print(f"Append random array of req shape:- {time() - er_t}")

    s = time()
    for _ in range(n_frames - 1):
        for _ in range(frame_step):
            ret, frame = src.read()
        if ret:
            ft_time = time()
            fr = format_frames(frame, output_size)
            result.append(fr)

        else:
            er_t = time()
            result.append(np.zeros_like(np.random.rand(224, 224, 3)))
            # print(f"Append random array of req shape:- {time() - er_t}")
                
    # print(f"Time for getting aligned face from n-1 frame:- {time() - s}")
    src.release()
    if len(result) < n_frames:
        print(len(result))
    
    result = np.array(result)[..., [2, 1, 0]]
    
    return result, label

In [80]:
n_frames = 30
batch_size = 8
AUTO = tf.data.AUTOTUNE



output_signature = (tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32, name="image"),
                    tf.TensorSpec(shape = (5,), dtype = tf.float32, name="labels"))

# # train_ds
# train_ds = tf.data.Dataset.from_generator(FrameGenerator(train_df, n_frames, training=True), (tf.float32, tf.float32))
# train_ds = train_ds.batch(batch_size)
# # train_ds = train_ds

# # val_ds
# val_ds = tf.data.Dataset.from_generator(FrameGenerator(test_df, n_frames, training=False), (tf.float32, tf.float32))
# val_ds = val_ds.batch(batch_size)

# # test data
# test_ds = tf.data.Dataset.from_generator(FrameGenerator(val_annotation, n_frames, training=False), (tf.float32, tf.float32))
# test_ds = test_ds.batch(batch_size)


# Used tf dataset from_generator to get metadata and map method to process video frames to decrease training time
train_ds = tf.data.Dataset.from_generator(MetaGenerator(train_df, n_frames, training=True), (tf.string, tf.float32))
train_ds = train_ds.map(lambda x,y: frames_from_video_file_map(x, y, 30), tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size).repeat()

val_ds = tf.data.Dataset.from_generator(MetaGenerator(test_df, n_frames, training=False), (tf.string, tf.float32))
val_ds = val_ds.map(lambda x,y: frames_from_video_file_map(x, y, 30), tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size)


# test data
test_ds = tf.data.Dataset.from_generator(MetaGenerator(val_annotation, n_frames, training=False), (tf.string, tf.float32))
test_ds = test_ds.map(lambda x,y: frames_from_video_file_map(x, y, 30),tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size)


In [81]:
# for ele in train_ds.take(1):
#     print(ele[1].numpy().shape)
#     break

In [82]:
checkpoint = MyModelCheckpoint(os.path.join(run_dir, 'env-personality-model-{epoch:02d}-{val_loss:.2f}.h5'),
        monitor='val_loss',verbose=1, save_best_only=True, mode='auto')
                              
callbacks_list = [checkpoint, reduce_lr]

In [83]:
# load trained model state
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

def load_model_data(model_path, opt_path):
    model = load_model(model_path)
    with open(opt_path, 'rb') as fp:
        d = pickle.load(fp)
        epoch = d['epoch']
        opt = d['opt']
    return epoch, model, opt

# epoch, model_env, opt = load_model_data('/mount1/harish/Personality_trait/checkpoint/00000-env_personality/env-personality-model-01-0.12.h5',
#                                         '/mount1/harish/Personality_trait/checkpoint/00000-env_personality/env-personality-model-01-0.12.pkl')

# model_env.compile(
#     loss = ['mae'],
#     # loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
#     optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), 
#     metrics = [mean_acc]
# )

In [84]:
model_env.fit(x= train_ds, steps_per_epoch=train_steps, 
              validation_data=val_ds, validation_steps=val_steps,
              epochs=20, callbacks=callbacks_list)

In [85]:
# model_env.predict(val_ds.take(1))

In [86]:
# initial code to train model

# model_env.fit(x= train_ds, steps_per_epoch=train_steps, validation_data=val_ds, 
#               validation_steps=val_steps,epochs=50, callbacks=callbacks_list)


In [94]:

# model = load_model(
#     "env_personality.h5",
#     custom_objects={"mean_acc": mean_acc}
#                   )

In [95]:
# pred = model.predict(test_ds)

In [96]:
# pred

In [97]:
# for x, y in test_ds:
#     print(y.numpy())